In [494]:
# import libraries:
import pandas as pd
import numpy as np
import random

In [513]:
# get inputs:
min_v = int(input("Enter the minimum value (min_v) you want to achieve: "))
max_w = int(input("Enter the maximum weight (max_w) allowed: "))

min_n = int(input("Enter the minimum number of snack types (min_n): "))
max_n = int(input("Enter the maximum number of snack types (max_n): "))
population_size = int(input("Enter population size: "))

Enter the minimum value (min_v) you want to achieve:  12
Enter the maximum weight (max_w) allowed:  10
Enter the minimum number of snack types (min_n):  2
Enter the maximum number of snack types (max_n):  4
Enter population size:  1000


In [514]:
# read CSV file:
file_path = 'snacks.csv'
df = pd.read_csv(file_path)


In [515]:
print(df)

            Snack  Available Weight  Value
0          MazMaz                10     10
1   Doogh-e-Abali                15     10
2            Nani                 5      5
3            Jooj                 7     15
4         Hot-Dog                20     15
5           Chips                 8      6
6        Nooshaba                12      8
7        Shokolat                 6      7
8       Chocoroll                 9     12
9         Cookies                11     11
10        Abnabat                 4      4
11   Adams-Khersi                14      9
12        Popcorn                16     13
13         Pastil                 3      7
14       Tordilla                10      9
15       Masghati                 5      6
16        Ghottab                 7     10
17   Saghe-Talaei                 9     11
18    Choob-Shoor                13     12


In [516]:
#Part One: Defining Basic Concepts

#Gene:
class Gene:
    def __init__(self, name, max_weight, value_per_weight):
        self.name = name
        self.weight = 0
        self.value_per_weight = value_per_weight
        self.max_weight = max_weight
    def __repr__(self):
        return f"Gene(name={self.name}, weight={self.weight}, max_weight={self.max_weight}, value_per_weight={self.value_per_weight})\n"

    def __str__(self):
        return f"{self.name}: Weight={self.weight}, Value per Weight={self.value_per_weight}"

# Genes pool:
genes_pool = df.copy()
genes_pool.rename(columns={'Snack': 'Name'}, inplace=True)
genes_pool.rename(columns={'Available Weight': 'Maximum Weight'}, inplace=True)
genes_pool['Value per Weight'] = genes_pool['Value'] / genes_pool['Maximum Weight']
genes_pool = genes_pool[['Name', 'Maximum Weight', 'Value per Weight']]

gene_objects = [Gene(row['Name'], row['Maximum Weight'], row['Value per Weight']) for index, row in genes_pool.iterrows()]

#Chromosome:
class Chromosome:
    def __init__(self, genes_pool, x):
        self.genes_temp = random.sample(genes_pool, x)
        self.genes = []
        for g in self.genes_temp:
            self.genes.append(Gene(g.name, g.max_weight, g.value_per_weight))
        for gene in self.genes:
            gene.weight = random.randint(0, gene.max_weight)
        self.total_weight = sum(gene.weight for gene in self.genes)
        self.total_value = sum(gene.weight * gene.value_per_weight for gene in self.genes)
        self.variety_of_snacks = len(self.genes)
        self.fitness = 0  
        
    def __repr__(self):
        return f"\nChromosome(genes=\n{self.genes}, fitness={self.fitness}) \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n"
        
    def __str__(self):
        genes_str = '\n'.join(str(gene) for gene in self.genes)
        return f"\nChromosome Details:\nGenes:\n{genes_str}\nFitness: {self.fitness} \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n"
        



In [517]:
print(genes_pool)

             Name  Maximum Weight  Value per Weight
0          MazMaz              10          1.000000
1   Doogh-e-Abali              15          0.666667
2            Nani               5          1.000000
3            Jooj               7          2.142857
4         Hot-Dog              20          0.750000
5           Chips               8          0.750000
6        Nooshaba              12          0.666667
7        Shokolat               6          1.166667
8       Chocoroll               9          1.333333
9         Cookies              11          1.000000
10        Abnabat               4          1.000000
11   Adams-Khersi              14          0.642857
12        Popcorn              16          0.812500
13         Pastil               3          2.333333
14       Tordilla              10          0.900000
15       Masghati               5          1.200000
16        Ghottab               7          1.428571
17   Saghe-Talaei               9          1.222222
18    Choob-

In [518]:
print(gene_objects)

[Gene(name=MazMaz, weight=0, max_weight=10, value_per_weight=1.0)
, Gene(name=Doogh-e-Abali, weight=0, max_weight=15, value_per_weight=0.6666666666666666)
, Gene(name=Nani, weight=0, max_weight=5, value_per_weight=1.0)
, Gene(name=Jooj, weight=0, max_weight=7, value_per_weight=2.142857142857143)
, Gene(name=Hot-Dog, weight=0, max_weight=20, value_per_weight=0.75)
, Gene(name=Chips, weight=0, max_weight=8, value_per_weight=0.75)
, Gene(name=Nooshaba, weight=0, max_weight=12, value_per_weight=0.6666666666666666)
, Gene(name=Shokolat, weight=0, max_weight=6, value_per_weight=1.1666666666666667)
, Gene(name=Chocoroll, weight=0, max_weight=9, value_per_weight=1.3333333333333333)
, Gene(name=Cookies, weight=0, max_weight=11, value_per_weight=1.0)
, Gene(name=Abnabat, weight=0, max_weight=4, value_per_weight=1.0)
, Gene(name=Adams-Khersi, weight=0, max_weight=14, value_per_weight=0.6428571428571429)
, Gene(name=Popcorn, weight=0, max_weight=16, value_per_weight=0.8125)
, Gene(name=Pastil, wei

In [519]:
#Part Two: Primary population production
#initial_population = [Chromosome(gene_objects, random.randint(min_n, max_n)) for _ in range(2)]
initial_population = []
for _ in range(population_size):
    c = Chromosome(gene_objects, random.randint(min_n, max_n))
    initial_population.append(c)
    c = None


In [520]:
initial_population

[
 Chromosome(genes=
 [Gene(name=Popcorn, weight=4, max_weight=16, value_per_weight=0.8125)
 , Gene(name=Choob-Shoor, weight=1, max_weight=13, value_per_weight=0.9230769230769231)
 , Gene(name=Hot-Dog, weight=9, max_weight=20, value_per_weight=0.75)
 , Gene(name=Ghottab, weight=2, max_weight=7, value_per_weight=1.4285714285714286)
 ], fitness=0) 
 Total Weight: 16
 Total Value: 13.780219780219781
 Range: 4,
 
 Chromosome(genes=
 [Gene(name=Nani, weight=5, max_weight=5, value_per_weight=1.0)
 , Gene(name=MazMaz, weight=10, max_weight=10, value_per_weight=1.0)
 , Gene(name=Ghottab, weight=2, max_weight=7, value_per_weight=1.4285714285714286)
 , Gene(name=Nooshaba, weight=5, max_weight=12, value_per_weight=0.6666666666666666)
 ], fitness=0) 
 Total Weight: 22
 Total Value: 21.19047619047619
 Range: 4,
 
 Chromosome(genes=
 [Gene(name=Masghati, weight=3, max_weight=5, value_per_weight=1.2)
 , Gene(name=Nooshaba, weight=0, max_weight=12, value_per_weight=0.6666666666666666)
 , Gene(name=Cho

In [521]:
#Part three: Implementation and specification of compatibility criterion function


def calculate_fitness(chromosome):
    fitness = chromosome.total_value
    penalty_weight =max_w - chromosome.total_weight
    penalty_value = chromosome.total_value - min_v
    #penalty_varity = min(chromosome.variety_of_snacks - max_n , min_n - chromosome.variety_of_snacks)
    penalties = 0
    if penalty_weight < 0:
        penalties = penalty_weight
    if penalty_value < 0:
        penalties = penalties + penalty_value
    if penalties < 0 :
        fitness = penalties
    return fitness
    


In [522]:
#Update Chromosomes Fitness'

for Chromosome in initial_population:
    Chromosome.fitness = calculate_fitness(Chromosome)

In [523]:
initial_population

[
 Chromosome(genes=
 [Gene(name=Popcorn, weight=4, max_weight=16, value_per_weight=0.8125)
 , Gene(name=Choob-Shoor, weight=1, max_weight=13, value_per_weight=0.9230769230769231)
 , Gene(name=Hot-Dog, weight=9, max_weight=20, value_per_weight=0.75)
 , Gene(name=Ghottab, weight=2, max_weight=7, value_per_weight=1.4285714285714286)
 ], fitness=-6) 
 Total Weight: 16
 Total Value: 13.780219780219781
 Range: 4,
 
 Chromosome(genes=
 [Gene(name=Nani, weight=5, max_weight=5, value_per_weight=1.0)
 , Gene(name=MazMaz, weight=10, max_weight=10, value_per_weight=1.0)
 , Gene(name=Ghottab, weight=2, max_weight=7, value_per_weight=1.4285714285714286)
 , Gene(name=Nooshaba, weight=5, max_weight=12, value_per_weight=0.6666666666666666)
 ], fitness=-12) 
 Total Weight: 22
 Total Value: 21.19047619047619
 Range: 4,
 
 Chromosome(genes=
 [Gene(name=Masghati, weight=3, max_weight=5, value_per_weight=1.2)
 , Gene(name=Nooshaba, weight=0, max_weight=12, value_per_weight=0.6666666666666666)
 , Gene(name=

In [524]:
def find_winner(population):
    max_fitness = float('-inf')
    winner = population[0];
    for Chromosome in population:
        if (Chromosome.fitness > max_fitness):
            max_fitness = Chromosome.fitness
            winner = Chromosome
    return winner

In [525]:
def check_for_answer(population):
    winner = find_winner(population)
    if winner.fitness >= min_v:
        #print("answer: ")
        #print(winner)
        #return True
        return winner
    #print("not found!")
    #return False
    return None

In [526]:
#Part four: Generating a new generation

#Crossover:
def crossover(population):
    new_population = population
    return new_population

#Mutation:
def mutation(population):
    new_population = population
    return new_population

In [527]:
population = initial_population
winner = check_for_answer(initial_population)
counter = 0
while check_for_answer(population) == False:
    new_population = crossover(population)
    new_population = mutation(new_population)
    population = new_population
    winner = check_for_answer(population)
    counter = counter + 1
    

In [528]:
def print_winner(winner):
    for Gene in winner.genes:
        print(f"{Gene.name}: {Gene.weight}")
    print(f"Total Weight: {winner.total_weight}")
    print(f"Total Value: {winner.total_value}")

In [529]:
print_winner(winner)

Chips: 1
Nani: 0
Pastil: 3
Jooj: 5
Total Weight: 9
Total Value: 18.464285714285715


In [530]:
winner



Chromosome(genes=
[Gene(name=Chips, weight=1, max_weight=8, value_per_weight=0.75)
, Gene(name=Nani, weight=0, max_weight=5, value_per_weight=1.0)
, Gene(name=Pastil, weight=3, max_weight=3, value_per_weight=2.3333333333333335)
, Gene(name=Jooj, weight=5, max_weight=7, value_per_weight=2.142857142857143)
], fitness=18.464285714285715) 
Total Weight: 9
Total Value: 18.464285714285715
Range: 4